# Машинное обучение для текстов.

### Общая информация:
Нам нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучим модель классифицировать комментарии на позитивные и негативные со значением метрики качества F1 не меньше 0.75.

### План работ:
Выполнить проект без BERT .

Загрузим и подготовим данные.
Обучим разные модели.
Сделайтем выводы.

### Описание датасета:
text- текст комментария, 

toxic — целевой признак.

### Содержание:

<a href='#1'>1.Рассмотрим и подготовим данные</a>

<a href='#2'>2.Обучим разные модели.</a>

<a href='#3'>3.Вывод.</a>


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import warnings
from tqdm.notebook import tqdm
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoost
from sklearn.linear_model import LinearRegression
import re
import transformers
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
m = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\litvinskaya.m\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\litvinskaya.m\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def lemmatize(text):
    txt=m.lemmatize(text)
    text="".join(txt) 
    return text

In [3]:
def clear_text(text):
    txt=re.sub(r'[^a-zA-Z]',' ', text)
    txt=txt.split() 
    txt=" ".join(txt)
    return txt

In [4]:
stop=set(stopwords.words('english'))

In [5]:
try:
    df=pd.read_csv('/datasets/toxic_comments.csv')
except:
    df=pd.read_csv(r'C:\Users\litvinskaya.m\Desktop\Материалы курса Яндекс.Практикум\ноутбуки\спринт 3.проект 4\toxic_comments.csv')


# 1.Рассмотрим и подготовим данные
 <a id='1'></a>

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [7]:
df.sample(3)

,text,toxic
63437,Would it be possible to merge these two aricle...,0
19192,Again - it is obvious what is happening. I've ...,0
14063,Amended rationale to specify image usage.,0


In [8]:
df['text']=df['text'].apply(clear_text)

In [9]:
df['text']=df['text'].apply(lemmatize)

In [10]:
features=df['text']
target=df['toxic']

In [11]:
features_train,features_test,target_train,target_test=train_test_split(features,
                                          target,test_size=0.25,random_state=42)

In [12]:
count_vect=CountVectorizer(stop_words=stop)
features=count_vect.fit_transform(features)

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stop) 
features_train=count_tf_idf.fit_transform(features_train)
features_test=count_tf_idf.transform(features_test) 

# 1.Обучим разные модели.
 <a id='2'></a>

In [14]:
%%time
print('DummyRegressor')
dummy_model= DummyRegressor(strategy="median")
dummy_model.fit(features_train,target_train)
dummy_result=dummy_model.predict(features_test)
dummy_result=pd.Series(dummy_result).astype('float')
print(f1_score(dummy_result,target_test))


DummyRegressor
0.0
Wall time: 29.9 ms


<div class="alert alert-block alert-info">DummyRegressor всегда нужен для проверки остальных моделей на адекватность</div>

In [15]:
%%time
print('LogisticRegression')
LRmodel=None
best_result=0

for sol in tqdm(['newton-cg','lbfgs','liblinear','sag','saga'], desc='solver', leave=False):
    model=LogisticRegression(random_state=42,solver=sol)
    model.fit(features_train,target_train)
    predict=model.predict(features_train)
    predict=pd.Series(predict).astype('float')
    result=f1_score(predict,target_train)
    if result>best_result:
        LRmodel=model

LogisticRegression


Wall time: 17.5 s


In [16]:
predict=LRmodel.predict(features_test)
print(f1_score(predict,target_test))

0.7367792052584404


In [17]:
%%time
print('LinearRegression')
model_Regression=LinearRegression()
model_Regression.fit(features_train,target_train)
predict=model_Regression.predict(features_test)
predict=pd.Series(predict).astype('float')
#print(f1_score(target_test,predict))

LinearRegression
Wall time: 10min 6s


In [ ]:
%%time
print('DecisionTreeClassifier')
best_model_Baum=None
best_result=0
best_depth=0
for depth in range(1,100,3):
    model_Baum=DecisionTreeClassifier(random_state=42,max_depth=depth)
    model_Baum.fit(features_train,target_train)
    result=cross_val_score(model_Baum, features_train, target_train, cv=5).mean()
    if result>best_result:
        best_model_Baum=model_Baum
        best_result=result
        best_depth=depth
print(best_result)

predict=model_Baum.predict(features_test)
print(f1_score(target_test,predict))

In [ ]:
#%%time
#print('CatBoostClassifier')
#model_Cat =CatBoostClassifier(iterations=50,verbose=10)
#features_train_f_cb=features_train.toarray()
#target_train_f_cb=target_train.toarray()
#features_test_f_cb=features_test.toarray()
#target_test_f_cb=target_test.toarray()
#model_Cat.fit(features_train,target_train)
#result=cross_val_score(model_Cat, features_train_f_cb, target_train_f_cb, cv=3).mean()
#predict=model_Cat.predict(features_test)
#predict=pd.Series(predict).astype('int')
#print(f1_score(target_test,predict))

In [ ]:
#%%time
print('LGBMRegressor')
model_GBM=LGBMRegressor(class_weight='balanced')
model_GBM.fit(features_train,target_train)
prediction_GBM= model_GBM.predict(features_test)
prediction_GBM=pd.Series(prediction_GBM).astype('int')
result=f1_score(target_test,prediction_GBM)
print(result)

In [ ]:

print('LGBMClassifier')
model_GBMc=LGBMClassifier(random_state=42,n_estimators=500,boosting_type="goss",max_depth=35)
model_GBMc.fit(features_train,target_train)
prediction_GBMc= model_GBMc.predict(features_test)
prediction_GBMc=pd.Series(prediction_GBMc).astype('int')
result=f1_score(target_test,prediction_GBMc)
print(result)

# 3.Вывод
 <a id='3'></a>


Мощностей линейной модели не хватает.
    

Логистическая модель приятно удивляет скоростью бучения в 22 секунды, но к сожалению, не достягивает до поставленной нами цели в F1=0.75
    

DecisionTreeClassifier справляется, но работает долго и не достаточно качественно,что делает его даже хуже Логистической модели в нашем конкретном случае.
    
В связи с тем,что Дерево оказалось медленным и неэффективным было принято решение пропустить темтирование случайного леса, и перейти к другим моделям.
    

Лучше всех нам подошел LGBMClassifier c результатом f1=0.78
    

Отдельно хочется отметить что с таким объемом данных мы работает впервые, поэтому нужно запастись терпением.</div>